In [ ]:
# This code file will run the ingest scripts
# 1 - Insert all the raw json from the database
# 2 - Convert these into datajoint tables
# 3 - Eventually, run automated scripts to do basic analyses

In [9]:
import datajoint as dj
import json
import math
import collections
import os.path as path

In [35]:
# Connect to the datajoint server
with open('.env','r') as cid:
    for line in cid:
        key,val = line.partition('=')[::2]
        dj.config[key] = val.rstrip('\n')

In [14]:
# Load the json file
with open('data/alyx_full.json','r') as fid:
    keys = json.load(fid)
# remove invalid uuid from unused tables
keys = [key for key in keys
        if key['model'] not in
        ['auth.group', 'sessions.session', 'authtoken.token']]


In [38]:
# Generate a raw schema

schema = dj.schema('testing_alyxraw')

@schema
class AlyxRaw(dj.Manual):
    definition = '''
    uuid: uuid  # pk field (uuid string repr)
    ---
    model: varchar(255)  # alyx 'model'
    '''

    class Field(dj.Part):
        definition = '''
        -> master
        fname: varchar(255)  # field name
        value_idx: tinyint
        ---
        fvalue=null: varchar(40000)  # field value in the position of value_idx
        index (fname)
        '''


In [39]:
alyxraw = AlyxRaw()